## Map state/block

In [305]:
from enum import Enum
import itertools
import numpy as np
import pandas as pd
from typing import Set, List, Tuple
class MapState(Enum):
    blank = 0 # can be anything here
    structure = 1 # confirmed complete structure
    hit = 7 # unconfirmed if has structure
    hit_confirmed = 8 # confirmed has structure with a hit (a structure has to be here but blocksize is unknown) could also be a decoy
    hit_miss = 9 # confirmed no structure (structure can't be here)
    decoy = 2 # decoy is put down


In [306]:
class MapBlock:
    def __init__(self, dimx:int, dimy: int, is_decoy:bool = False):
        self.x = dimx
        self.y = dimy
        self.is_decoy = is_decoy

    def __str__(self) -> str:
        return f"{self.x} x {self.y} {'decoy' if self.is_decoy else ''}"
    
    def __repr__(self):
        return self.__str__()

    def __hash__(self) -> int:
        return hash((self.x, self.y))
    
    def rotate(self, direction: int):
        if self.x==self.y:
            return
         
        dimx, dimy = self.x, self.y
        self.x = dimx if dimx < dimy else dimy
        self.y = dimy if dimx < dimy else dimx
        if direction == 1:
            self.x, self.y = self.y, self.x


## Map

In [428]:

class BattleshipMap:
    def __init__(self, width = 10, height = 10):
        # blank map!
        self.width = width
        self.height = height
        self.map  = [ [MapState.blank.value for _ in range(width)] for _ in range(height)]  
        self.confirmed_hits: Set = set()
    
    def __str__(self) -> str:
        ylabel = [chr(i) for i in range(74,64,-1)]

        return_str = ''
        for i ,row in enumerate(self.map):
            return_str += f"{ylabel[i]} {str(row)}\n"
        xlabel = ", ".join(map(str, (range(1,11))))
        return_str += f"   {xlabel} "
        return return_str
    
    def _is_oob(self, x: int, y:int):
        """is coords out of bounds

        Args:
            x (int): _description_
            y (int): _description_

        Returns:
            _type_: _description_
        """
        if x < 0 or y < 0 or x > self.height or y > self.width:
            return True
        return False
    

    def is_droppable(self, x:int, y:int, block: MapBlock) -> bool:
        """Can a block be dropped at this coordinate
        - if any coord in the block is structure/confirmed miss, can't drop
        - if all coords in the block is hit, can't drop, unless it's a decoy

        Args:
            x (int): _description_
            y (int): _description_
            block (MapBlock): _description_

        Returns:
            bool: _description_
        """
        if self._is_oob(x+block.x, y+block.y):
            return False  

        # get all coords this block occupies
        coords = list(itertools.product(
            range(x, x+block.x), 
            range(y, y+block.y)))

        all_hit = True
        for x,y in coords:
            if ( self.is_state(x,y, MapState.structure) or # if there's other structure
                self.is_state(x,y, MapState.hit_miss) # confirmed empty
            ):
                return False # occupied or confirmed empty, go next 
            
            # check if the entire place has been hit, then we can't put a map block 
            all_hit = all_hit and self.is_state(x,y,MapState.hit)
        
        # TODO, but can we put decoys?
        # Assumption is that we don't know other teams hit a decoy; so decoys can be where map is hit
        # Else, decoys can't be where map is hit by other team and remove last bool statement 
        if all_hit and block.is_decoy == False:
            return False
        
        return True


    def get_confirmed_hits(self):
        return self.confirmed_hits
    

    def get_constrained_drop_coords(self, block: MapBlock, constraints: List[Tuple[int]]) -> List[Tuple[int]]: 
        """find all coordinate where a map block can be dropped and covers constraints

        Args:
            block (MapBlock): _description_
            constraints (List[Tuple[int]]): _description_

        Returns:
            List[Tuple[int]]: _description_
        """ 
        # the box has to cover all coords in constraints
        setx = set(range(self.height))
        sety = set(range(self.width))

        for x,y in constraints:
            setx = setx.intersection(set(range( x - block.x+1, x+1)))
            sety = sety.intersection(set(range( y - block.y+1, y+1))) 
         
        if not setx or not sety:
            return []
        
        # all coords where block can be with the constraint
        coords = list(itertools.product(setx, sety)) 
        
        return [c for c in coords if self.is_droppable(*c, block)]

    def get_drop_coords(self, block: MapBlock) -> list:
        """find all coordinate where a map block can be dropped

        Args:
            block (MapBlock): _description_
        Returns:
            list: _description_
        """
        droppable_coords = list()
        for i in range(self.height):
            for j in range(self.width):
                # can we drop a block at i,j ? 
                if self.is_droppable(i, j, block):
                    droppable_coords.append((i,j))

        return droppable_coords
                
 
    def drop_block(self, x:int, y:int, block: MapBlock):  
        """Drop a block, assumes it's a legal location

        Args:
            x (int): _description_
            y (int): _description_
            block (MapBlock): _description_
        """

        coords = list(itertools.product(
            range(x, x+block.x) , 
            range(y, y+block.y)
            ))
        
        for i, j in coords:
            state = MapState.structure if not block.is_decoy else MapState.decoy
            self.set_state(i, j, state)


    def get_normalized_map(self) -> np.array:
        """get normalized map as a numpy array
        change MapState.hit and MapState.decoy to 0 as they are treated as no building

        Returns:
            np.array: _description_
        """
        # the confirmed hits should be replaced by a building 
        assert len(self.confirmed_hits) == 0

        np_map = np.array(self.map)
        np_map = np.where(
            (np_map ==MapState.hit.value) 
            |(np_map ==MapState.decoy.value)
            |(np_map ==MapState.hit_miss.value), 0, np_map)
        return np_map

    def set_state(self, x:int, y:int, map_state: MapState):
        if self._is_oob(x,y):
            print("out of bounds!") # honestly this shouldnt happen
            return -1 
        
        if map_state == MapState.hit_confirmed:
            self.confirmed_hits.add((x,y))
        elif self.get_state(x,y) == MapState.hit_confirmed and map_state != MapState.hit_confirmed:
            self.confirmed_hits.remove((x,y))

        self.map[x][y] = map_state.value 

    def is_state(self, x:int, y:int, map_state: MapState):
        return self.map[x][y] == map_state.value
    
    def get_state(self, x:int, y:int) -> MapState:
        return MapState(self.map[x][y])
    
    def __hash__(self) -> int:
        return hash(tuple(tuple(row) for row in self.map))

    def __repr__(self):
        return self.__str__()




In [429]:
# default map/blocks
tmap= BattleshipMap()
for i, state in enumerate(MapState):
    tmap.set_state(0,i, state) 
tmap.set_state(0, 3, MapState.hit) 

tmap, tmap.get_normalized_map()



(J [0, 1, 7, 7, 9, 2, 0, 0, 0, 0]
 I [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 H [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 G [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 F [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 E [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 D [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 C [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 B [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 A [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ,
 array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

In [430]:
# blocks
block_sizes = [(2,2)] * 5 + [(3,3)]  + [(1,3)]*3
decoy_sizes = [(1,1)]*4
blocks = [MapBlock(*size) for size in block_sizes]  + [MapBlock(*size, is_decoy=True) for size in decoy_sizes]  

## Map generator 

In [431]:

import itertools
from copy import deepcopy

In [432]:

all_maps = list()
past_blocks_data = set()

def fit_block(bmap: BattleshipMap, blocks: list[MapBlock]):
    if len(blocks) == 0:
        # finished putting 
        # print("Finished blocks!")
        map_cpy = deepcopy(bmap)  
        all_maps.append(map_cpy)
        return

    block: MapBlock = blocks.pop()

    for i in range(bmap.height):
        for j in range(bmap.width):
            # drop a block at i,j 
            if not bmap.is_droppable(i,j, block):
                continue

            # get all coords this block occupies
            coords = list(itertools.product(range(i, i+block.x) , range(j, j+block.y))) 
            
            # its droppable
            # keep track of old state
            old_state = {c: bmap.get_state(c[0], c[1]) for c in coords}

            # drop block on map
            bmap.drop_block(i, j, block) 


            # print(f"dropping block {block} at {(i,j)} with {coords}")
            # print(template)

            # memoization
            if (bmap, tuple(blocks)) in past_blocks_data:
                for x,y in coords:
                    bmap.set_state(x,y, old_state[(x,y)]) 
                continue
            # memoization
            past_blocks_data.add( (bmap, tuple(blocks)))

            fit_block(bmap, blocks)

            # return to original state
            for x,y in coords:
                bmap.set_state(x,y, old_state[(x,y)]) 
    
    blocks.append(block)


## trial run for generator

In [440]:
# vahumana map
trial_map = BattleshipMap()
for i in range(4,7):
    for j in range(1,4):
        trial_map.set_state(i,j, MapState.structure)
        
map_blocks = [(6,6),[4,4],[4,4]]

decoy_blocks = []
trial_blocks = [MapBlock(*size) for size in map_blocks]  + [MapBlock(*size, is_decoy=True) for size in decoy_blocks] 
trial_blocks

all_maps = list()
past_blocks_data = set()
fit_block(trial_map, trial_blocks) 


In [434]:
len(all_maps)

9

## Coord conversion

In [451]:
9-(-65), 66+8, 67+7

(74, 74, 74)

In [452]:
def convert_coords(map_coord: tuple):
    """convert map coord (A,1) to array coord (9,0)

    Args:
        map_coord (tuple): _description_
    """ 
    return -ord(map_coord[0])+74, map_coord[1] - 1
    

## Set up vahumana map

In [314]:
# vahumana map
vahumana = BattleshipMap()
for i in range(4,7):
    for j in range(1,4):
        vahumana.set_state(i,j, MapState.structure)

hit_coords = [(7,2), (8, 1), (6,7),(2,7), (0,8), (1,8), (2,8)]
for x,y in hit_coords:
    vahumana.set_state(x,y, MapState.hit)
print(vahumana)

J [0, 0, 0, 0, 0, 0, 0, 0, 7, 0]
I [0, 0, 0, 0, 0, 0, 0, 0, 7, 0]
H [0, 0, 0, 0, 0, 0, 0, 7, 7, 0]
G [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
F [0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
E [0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
D [0, 1, 1, 1, 0, 0, 0, 7, 0, 0]
C [0, 0, 7, 0, 0, 0, 0, 0, 0, 0]
B [0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
A [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
   1, 2, 3, 4, 5, 6, 7, 8, 9, 10 


In [315]:
map_blocks = [(2,2)] * 5  + [(1,3)]*3
# map_blocks =  [(1,3)]*3
decoy_blocks = [(1,1)]*4 
# decoy_blocks = []
vahumana_blocks = [MapBlock(*size) for size in map_blocks]  + [MapBlock(*size, is_decoy=True) for size in decoy_blocks] 
print(vahumana_blocks)

[2 x 2 , 2 x 2 , 2 x 2 , 2 x 2 , 2 x 2 , 1 x 3 , 1 x 3 , 1 x 3 , 1 x 1 decoy, 1 x 1 decoy, 1 x 1 decoy, 1 x 1 decoy]


## set up amurta

In [476]:
# amurta map
amurta = BattleshipMap()
for i in range(5,6):
    for j in range(3,6):
        amurta.set_state(i,j, MapState.structure)

map_hit = [('G',4), ('G',7), ('G',8), ('G', 10),
           ('H',8), ('H',9), ('I',9),
           ('C',6), ('C',8), ('B',4), ('A',6)]
amurta_hit_coords = [convert_coords(h) for h in map_hit]
print(amurta_hit_coords)

for x,y in amurta_hit_coords:
    amurta.set_state(x,y, MapState.hit)
print(amurta)


[(3, 3), (3, 6), (3, 7), (3, 9), (2, 7), (2, 8), (1, 8), (7, 5), (7, 7), (8, 3), (9, 5)]
J [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
I [0, 0, 0, 0, 0, 0, 0, 0, 7, 0]
H [0, 0, 0, 0, 0, 0, 0, 7, 7, 0]
G [0, 0, 0, 7, 0, 0, 7, 7, 0, 7]
F [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
E [0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
D [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C [0, 0, 0, 0, 0, 7, 0, 7, 0, 0]
B [0, 0, 0, 7, 0, 0, 0, 0, 0, 0]
A [0, 0, 0, 0, 0, 7, 0, 0, 0, 0]
   1, 2, 3, 4, 5, 6, 7, 8, 9, 10 


In [455]:
map_blocks = [(3,3)]*1 + [(2,2)] * 5  + [(1,3)]*2
# map_blocks =  [(1,3)]*3
decoy_blocks = [(1,1)]*4 
# decoy_blocks = []
amurta_blocks = [MapBlock(*size) for size in map_blocks]  + [MapBlock(*size, is_decoy=True) for size in decoy_blocks] 
print(amurta_blocks)

[3 x 3 , 2 x 2 , 2 x 2 , 2 x 2 , 2 x 2 , 2 x 2 , 1 x 3 , 1 x 3 , 1 x 1 decoy, 1 x 1 decoy, 1 x 1 decoy, 1 x 1 decoy]


## randomization

In [441]:
import random

def group_confirmed_hits(confirmed_hits: Set[Tuple]):

    # TODO we'll think about a block covering two confirmed hits later 
    # we dont need to worry about that as long as we update confirmed hits..
    for c in confirmed_hits:
        yield [c]
        
def make_random_map(bmap: BattleshipMap, blocks: list[MapBlock], num_blocks: int):
    assert num_blocks <= len(blocks)

    randomized_blocks = random.sample(blocks, num_blocks) 

    confirmed_hits = list(bmap.get_confirmed_hits())

    # if there's confirmed hits in the map, we have to place a block there 
    # we dropping first N block in N confirmed hits: 
    for i, constraint in enumerate(confirmed_hits): 
        # if confirmed hits change..
        if constraint not in bmap.get_confirmed_hits():
            continue

        block = randomized_blocks[i]
        # get coords where we can drop the block
        drop_coords = bmap.get_constrained_drop_coords(block, [constraint])
        # print("constraint", constraint, "droppable", drop_coords, "block dim", block)

        # pick a random one and drop
        drop_coord = random.choice(drop_coords)
        bmap.drop_block(*drop_coord, block)
        # print(block)

    for block in randomized_blocks[len(confirmed_hits):]:
        # TODO, 
        # coin toss to determine block direction
        direction = random.randint(0,1)
        block.rotate(direction)
        
        drop_coord = None
        # selection 1 random
        for _ in range(5):
            x = random.randint(0, bmap.height - 1)
            y = random.randint(0, bmap.width - 1)
            if bmap.is_droppable(x,y, block):
                drop_coord = x,y
                break
        if not drop_coord:
            # selection 2, get all viable
            viable_coords = bmap.get_drop_coords(block)
            if not viable_coords:
                return None
            drop_coord = random.choice(viable_coords)

        bmap.drop_block(*drop_coord, block)

    return bmap
    


### amurta

In [458]:
## mark hits
hit_miss_map = [('C',8)]
hit_miss_coords = [convert_coords(h) for h in hit_miss_map]
print(hit_miss_coords)

for coords in hit_miss_coords:
    amurta.set_state(*coords , MapState.hit_miss)
# amurta.set_state(7, 7 , MapState.blank)
amurta

[(7, 7)]


J [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
I [0, 0, 0, 0, 0, 0, 0, 0, 7, 0]
H [0, 0, 0, 0, 0, 0, 0, 7, 7, 0]
G [0, 0, 0, 7, 0, 0, 7, 7, 0, 7]
F [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
E [0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
D [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C [0, 0, 0, 0, 0, 7, 0, 9, 0, 0]
B [0, 0, 0, 7, 0, 0, 0, 0, 0, 0]
A [0, 0, 0, 0, 0, 7, 0, 0, 0, 0]
   1, 2, 3, 4, 5, 6, 7, 8, 9, 10 

In [463]:

import numpy as np  
from tqdm import tqdm
randomized_maps: list[BattleshipMap] = list()
normalized_maps = list()


num_maps = 10**6 #10**6
num_maps = 5*10**5
# num_maps = 10
for i in tqdm(range(num_maps)):
    res = make_random_map(deepcopy(amurta), amurta_blocks, len(amurta_blocks))
    if res:
        # randomized_maps.append(res)
        # print(res)
        normalized_maps.append(res.get_normalized_map())

# 1000 = 0.2 
# 100,000 = 25.9s
# normalize probabllity (7/2 mark as 0) 

100%|██████████| 500000/500000 [04:02<00:00, 2063.17it/s]


In [464]:
import pandas

prob_map = sum(normalized_maps)/len(normalized_maps)
with np.printoptions(precision=3):
    row_labels = [chr(i) for i in range(74,64,-1)]
    column_labels = map(str, range(1,11))
    df = pandas.DataFrame(prob_map, columns=column_labels, index=row_labels)
    # print(prob_map)

In [477]:
def _color_hit(df):
    color = 'background-color: #333333'
    df1 = pd.DataFrame('', index=df.index, columns=df.columns) 
    for c in amurta_hit_coords:
        df1.iloc[c] = color
    return df1 

def _color_red_or_green(val):
    color = '#75454f' if val < 0.48 or val >= 1 else 'green'
    return 'background-color: %s' % color

# style = df.style.format(precision=3).
style = df.style.format(precision=3).applymap(_color_red_or_green).apply(_color_hit, axis=None)
style

,1,2,3,4,5,6,7,8,9,10
J,0.155,0.270,0.295,0.284,0.283,0.284,0.283,0.295,0.271,0.157
I,0.272,0.460,0.480,0.460,0.460,0.461,0.460,0.481,0.461,0.273
H,0.299,0.487,0.525,0.508,0.511,0.510,0.501,0.519,0.482,0.297
G,0.294,0.460,0.489,0.448,0.458,0.450,0.462,0.485,0.465,0.285
F,0.302,0.466,0.419,0.270,0.279,0.270,0.393,0.501,0.484,0.290
E,0.309,0.444,0.292,1.000,1.000,1.000,0.260,0.448,0.468,0.276
D,0.301,0.463,0.416,0.267,0.287,0.274,0.311,0.274,0.376,0.269
C,0.305,0.480,0.514,0.472,0.499,0.461,0.277,0.000,0.231,0.238
B,0.273,0.464,0.490,0.480,0.498,0.473,0.378,0.230,0.352,0.246
A,0.157,0.271,0.294,0.282,0.289,0.273,0.267,0.236,0.245,0.135


In [332]:
df.to_csv('battleship_df/amurta/attack1.csv', index=False)

### vahumana

In [ ]:
 
randomized_maps: list[BattleshipMap] = list()
for i in range(2):
    res = make_random_map(deepcopy(vahumana), vahumana_blocks, len(vahumana_blocks))
    if res:
        randomized_maps.append( res)
# 1000 = 2.2s
# randomized
# 100,000 = 25.9s

In [ ]:
import numpy as np 
# normalize probabllity (7/2 mark as 0)
normalized_maps = list()
for m in randomized_maps:
    np_map = np.array(m.map)
    np_map = np.where((np_map==2) |(np_map==7), 0, np_map) 
    normalized_maps.append(np_map)

In [ ]:
prob_map = sum(normalized_maps)/2316602
with np.printoptions(precision=3):
    print(prob_map)
    print(np.array(range(1,11)))




[[0.146 0.249 0.261 0.258 0.258 0.259 0.259 0.264 0.231 0.149]
 [0.251 0.416 0.411 0.405 0.403 0.402 0.401 0.409 0.397 0.254]
 [0.276 0.412 0.41  0.4   0.41  0.399 0.396 0.403 0.391 0.266]
 [0.201 0.245 0.26  0.249 0.355 0.406 0.393 0.397 0.404 0.261]
 [0.092 1.    1.    1.    0.251 0.398 0.397 0.397 0.403 0.26 ]
 [0.093 1.    1.    1.    0.258 0.403 0.398 0.397 0.403 0.26 ]
 [0.092 1.    1.    1.    0.251 0.397 0.396 0.396 0.401 0.26 ]
 [0.206 0.252 0.265 0.254 0.359 0.409 0.397 0.4   0.407 0.263]
 [0.265 0.42  0.418 0.406 0.415 0.403 0.402 0.407 0.413 0.25 ]
 [0.149 0.255 0.267 0.263 0.259 0.258 0.258 0.262 0.25  0.147]]
[ 1  2  3  4  5  6  7  8  9 10]


In [ ]:
cleaned_prob = np.where((prob_map==1), 0, prob_map) 
max_index = cleaned_prob.argmax()
np.unravel_index(max_index, cleaned_prob.shape)

(8, 1)

In [ ]:
normalized_maps.argsort()[-3:][::-1]

array([[9, 0, 8, 1, 5, 6, 4, 7, 3, 2],
       [9, 0, 6, 5, 3, 7, 8, 4, 2, 1],
       [0, 1, 3, 9, 4, 6, 7, 8, 5, 2]])

### find all arrangement

In [ ]:
all_maps = list()
past_blocks_data = set()
fit_block(vahumana, vahumana_blocks) 
len(all_maps)


KeyboardInterrupt: 

In [ ]:
# find all if u wanna take the time la

In [32]:
for m in all_maps:
    print(m)
    break

J [1, 1, 0, 0, 0, 1, 1, 1, 1, 0]
I [1, 1, 0, 1, 1, 1, 1, 1, 1, 0]
H [0, 1, 1, 1, 1, 1, 1, 1, 1, 0]
G [0, 1, 1, 0, 0, 1, 1, 1, 1, 0]
F [0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
E [0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
D [0, 1, 1, 1, 0, 0, 1, 1, 0, 0]
C [0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
B [0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
A [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
   1, 2, 3, 4, 5, 6, 7, 8, 9, 10 
